In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob, Word, Blobber
import pandas as pd
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/twitter.csv")
Rev_tweet=data["text"]
Rev_tweet


0      We may not hit 90° today. \n\nMe: A bit bummed...
1      Join the CVSCareers team! See our latest job o...
2      See a virtual tour of my listing on 146 Buckin...
3      Lake Monticello has been selling fast and no w...
4      Lake Monticello has been selling fast and no w...
                             ...                        
921    Shades of #charlottesville https://t.co/b3sSnH...
922    @Lmo66Olson These White Nationalists look pret...
923    Want to know which #countertop #colors match y...
924    @kevinCash__ @TedToddAgency Didn't I see this ...
925    @senrobportman With the destruction your chose...
Name: text, Length: 926, dtype: object

In [3]:
def preprocess(text):
    clean_data = []
    for x in (text[:]): 
        new_text = re.sub('<.*?>', '', x)   # remove HTML tags
        new_text = re.sub(r'[^\w\s]', '', new_text) # remove punc.
        new_text = re.sub(r'\d+','',new_text)# remove numbers
        new_text = new_text.lower() # lower case, .upper() for upper
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags=re.UNICODE)
        new_text = emoji_pattern.sub(r'', new_text)         
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

In [4]:
data['clean_text']=preprocess(Rev_tweet)
data['clean_text']

0      we may not hit  today \n\nme a bit bummed if w...
1      join the cvscareers team see our latest job op...
2      see a virtual tour of my listing on  buckingha...
3      lake monticello has been selling fast and no w...
4      lake monticello has been selling fast and no w...
                             ...                        
921          shades of charlottesville httpstcobssnhfxyh
922    lmoolson these white nationalists look pretty ...
923    want to know which countertop colors match you...
924    kevincash__ tedtoddagency didnt i see this guy...
925    senrobportman with the destruction your chosen...
Name: clean_text, Length: 926, dtype: object

In [5]:
data['clean_tweets'] = data['clean_text'].str.replace("[^a-zA-Z#]", " ")
data['clean_tweets'] = data['clean_tweets'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
data['clean_tweets'] = data['clean_tweets'].fillna('').apply(lambda x: x.lower())

data['clean_tweets']

0      may not hit today bit bummed get close the rec...
1      join the cvscareers team see our latest job op...
2      see virtual tour listing buckingham cir charlo...
3      lake monticello has been selling fast and wond...
4      lake monticello has been selling fast and wond...
                             ...                        
921             shades charlottesville httpstcobssnhfxyh
922    lmoolson these white nationalists look pretty ...
923    want know which countertop colors match your c...
924    kevincash tedtoddagency didnt see this guy cha...
925    senrobportman with the destruction your chosen...
Name: clean_tweets, Length: 926, dtype: object

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# tokenization
tokenized_doc = data['clean_tweets'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# de-tokenization
detokenized_doc = []
for i in range(len(data)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
data['clean_tweets'] = detokenized_doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
data['clean_tweets']

0      may hit today bit bummed get close record migh...
1      join cvscareers team see latest job opening ht...
2      see virtual tour listing buckingham cir charlo...
3      lake monticello selling fast wonder great spot...
4      lake monticello selling fast wonder great spot...
                             ...                        
921             shades charlottesville httpstcobssnhfxyh
922    lmoolson white nationalists look pretty young ...
923    want know countertop colors match cabinets wel...
924    kevincash tedtoddagency didnt see guy charlott...
925    senrobportman destruction chosen leader trump ...
Name: clean_tweets, Length: 926, dtype: object

In [15]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(data['clean_tweets'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
X.shape

(926, 2871)

In [17]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)


In [19]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = data['clean_tweets']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,may hit today bit bummed get close record migh...,0.0067184300043487,0.0040112464170348
1,join cvscareers team see latest job opening ht...,0.0074926081765396,0.0044727463100097
2,see virtual tour listing buckingham cir charlo...,0.0070825412632500,0.0042367221235422
3,lake monticello selling fast wonder great spot...,0.0039653010337944,0.0024204555290122
4,lake monticello selling fast wonder great spot...,0.0039653010337944,0.0024204555290122
...,...,...,...
921,shades charlottesville httpstcobssnhfxyh,0.0120775468046511,0.0071818830352469
922,lmoolson white nationalists look pretty young ...,0.0071573686418166,0.0042292711792457
923,want know countertop colors match cabinets wel...,0.0044319048800804,0.0027675667264268
924,kevincash tedtoddagency didnt see guy charlott...,0.0078111775227258,0.0046873091003576


In [20]:
# Features or words used as features 
dictionary = vectorizer.get_feature_names()

In [21]:
dictionary

['abandon',
 'abbydphillip',
 'abc',
 'abetted',
 'able',
 'abundant',
 'abuseofpower',
 'academy',
 'accessibility',
 'accuse',
 'achievements',
 'aclu',
 'acosta',
 'acting',
 'action',
 'active',
 'activists',
 'actor',
 'actorslife',
 'actual',
 'addict',
 'adjourned',
 'adl',
 'adolf',
 'advantage',
 'advisories',
 'affection',
 'afford',
 'africans',
 'afternoon',
 'afterward',
 'againthis',
 'aggressive',
 'ago',
 'aide',
 'aided',
 'air',
 'aka',
 'albemarle',
 'alerts',
 'alike',
 'allday',
 'alllivesmatter',
 'alllivesmatters',
 'alongside',
 'alternative',
 'amandagoer',
 'amazing',
 'amazingview',
 'amber',
 'america',
 'american',
 'americans',
 'americorpsvista',
 'amers',
 'amp',
 'ampedup',
 'ams',
 'amy',
 'amyspitalnick',
 'analysis',
 'anarchists',
 'anarchy',
 'anchor',
 'android',
 'angela',
 'animals',
 'anniversary',
 'annual',
 'antifa',
 'antiracist',
 'antiracistaction',
 'antisemitic',
 'antisemitism',
 'antitrump',
 'aokay',
 'apparent',
 'apparently',
 'app

In [22]:
# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T

In [23]:
encoding_matrix

,topic_1,topic_2
abandon,0.0000059569291184,0.0000335978275831
abbydphillip,0.0000235006023462,0.0000177132911349
abc,0.0000080591169177,0.0000059333856478
abetted,0.0000070105219762,0.0000052401490812
able,0.0000108095748362,0.0000079336959569
...,...,...
yum,0.0000048385671911,0.0000035433033617
zavacky,0.0000119093945389,0.0000087206189341
zero,0.0000047779473694,0.0000034865673884
zilch,0.0000047779473694,0.0000034865673884
